In [2]:
%run Weighting.ipynb

Pixiedust database opened successfully


In [3]:
nilai_test = pd.read_csv("Splitted Test.csv", index_col = 0)

nilai_test

Nama Panggilan                         Asal Sekolah Jurusan Sekolah  \
157         Nadira  SMA Negeri 1 Kota Tangerang Selatan             IPS   
19           Nanda                      SMAN 60 Jakarta             IPA   
115          Rizki                      SMAN 97 JAKARTA             IPA   
46            Yogi                      SMAN 99 Jakarta             IPA   
119        Teresa                       Sman 20 Bandung             IPS   
58            Alfa                        SMAN 4 BEKASI             IPS   
175           Dini                      Sman 35 jakarta             IPA   
171            Sam                     SMAN 109 Jakarta             IPA   
181          Nivea                      SMAN 98 Jakarta             IPA   
179          febby                      sman 62 jakarta             IPA   
99           Yogie                  SMA Negeri 2 Bangko             IPA   
167         Azkiya                         SMAN 1 Depok             IPS   
44            Afni                         SMAN 4 DEPOK             IPA   
101            Isa             SMA Kesatrian 1 Semarang             IPA   
169         Nethan                     SMAN 101 Jakarta             IPS   
40           Salsa                     SMAN 113 Jakarta             IPA   
99           Yogie                  SMA Negeri 2 Bangko             IPA   
75          Safmor             SMAN 2 Tangernag Selatan             IPA   
82           Nanda                         SMAN 3 Depok             IPA   
122           Wuri                      SMAN 99 Jakarta             IPA   
170           Hana                     SMAN 106 JAKARTA             IPA   
114           Heno                       SMAN 1 Babelan             IPA   
8             Rais                      SMAN 48 Jakarta             IPA   
106            TIM                     SMAN 112 JAKARTA             IPA   
180            Ita                      SMAN 82 JAKARTA             IPS   
165          Firda                SMA Negeri 65 Jakarta             IPA   
17           Salsa                      SMAN 70 Jakarta             IPA   
132          Fiqih                    SMAN 106 Jakartaa             IPS   
109          Linda                       SMAN 1 JAKARTA             IPS   
29            Zaza               SMAS Nurul Fikri Depok             IPS   
123          Naura                             Sman 60              IPS   
125           Ahya                      SMAN 51 jakarta             IPA   
91            Nida                 SMAN 1 Kota Sukabumi             IPA   
53            Alia                      Sma bunda mulia             IPA   
29            Zaza               SMAS Nurul Fikri Depok             IPS   
182          Wahyu                      SMAN 98 JAKARTA             IPA   
40           Salsa                     SMAN 113 Jakarta             IPA   
64           fitri                      sman 99 jakarta             IPS   
65           Kevin                    SMAN 2 PANGKAJENE             IPA   
170           Hana                     SMAN 106 JAKARTA             IPA   
157         Nadira  SMA Negeri 1 Kota Tangerang Selatan             IPS   
7            fitri                      sman 2 cibinong             IPA   
9            Dinda                SMA Negeri 39 Jakarta             IPA   
124           Wima                      SMAN 97 Jakarta             IPS   
69           Fitri                 SMAN 1 Rangkasbitung             IPA   
84           Dicky                      SMAN 99 Jakarta             IPA   
180            Ita                      SMAN 82 JAKARTA             IPS   
120           faiq                              sman 60             IPS   
58            Alfa                        SMAN 4 BEKASI             IPS   
42            Luta                      SMAN 68 Jakarta             IPS   
167         Azkiya                         SMAN 1 Depok             IPS   
97          Rismat                 SMAN 1 Kota Sukabumi             IPA   
124           Wima           

In [ ]:
tabel_alternatif = open_tabel("Tabel Alternatif.csv")

tabel_alternatif

In [ ]:
pilihan_jurusan = get_jurusan(tabel_alternatif)

pilihan_jurusan

In [ ]:
mean = get_mean(tabel_alternatif)
std = get_std(tabel_alternatif)

In [ ]:
bener = 0
aw = list()
for test_index in nilai_test.index:
#assign nilai IPA/IPS
    try:
        if nilai_test["Jurusan Sekolah"][test_index].casefold() == "ipa":
            nilai = [
                nilai_test["Rata-rata Matematika"][test_index],
                nilai_test["Rata-rata Inggris"][test_index],
                nilai_test["Rata-rata Indonesia"][test_index],
                nilai_test["Rata-rata Fisika"][test_index],
                nilai_test["Rata-rata Kimia"][test_index],
                nilai_test["Rata-rata Biologi"][test_index],
                0,
                0,
                0,
            ]
        elif nilai_test["Jurusan Sekolah"][test_index].casefold() == "ips":
            nilai = [
                0,
                nilai_test["Rata-rata Inggris"][test_index],
                nilai_test["Rata-rata Indonesia"][test_index],
                0,
                0,
                0,
                nilai_test["Rata-rata Ekonomi"][test_index],
                nilai_test["Rata-rata Geografi"][test_index],
                nilai_test["Rata-rata Sosiologi"][test_index],
            ]
    except AttributeError:
        continue

    #Mendapatkan nilai2 parameter yang hendak digunakan
    params = get_params(mean, nilai, std)

    #Menambahkan kolom parameter ke tabel_alternatif
    tabel_alternatif = add_parameter(params, tabel_alternatif)

    #Mencari nilai fungsi preferensi gaussian
    fp_gaussian = fungsi_preferensi_gaussian(tabel_alternatif)

    #Menghitung index preferensi global
    index_preferensi_global = idx_pref_global1(fp_gaussian, f"" ,weight)

    #Menghitung entering flow
    e_flow = entering_flow(pilihan_jurusan, index_preferensi_global)

    #Menghitung leaving flow
    l_flow = leaving_flow(pilihan_jurusan, index_preferensi_global)

    #Menghitung net flow
    n_flow = net_flow(pilihan_jurusan, e_flow, l_flow)

    #Mendapatkan Hasil Outrank
    hasil_outrank = sorted(n_flow.items(), key = lambda kv:(kv[1], kv[0]), reverse = True)

    tab = dict()

    rank = list()
    jurusan = list()
    net = list()

    for i in range(len(hasil_outrank)):
        rank.append(i+1)
        jurusan.append(hasil_outrank[i][0])
        net.append(hasil_outrank[i][1])

    tab["Jurusan"] = jurusan
    tab["Net Flow"] = net

    tabel_hasil = pd.DataFrame(tab, index=rank)
#     aw.append((tabel_hasil["Jurusan"][1], tabel_hasil["Jurusan"][2], tabel_hasil["Jurusan"][3], tabel_hasil["Jurusan"][4], tabel_hasil["Jurusan"][5], tabel_hasil["Jurusan"][6], tabel_hasil["Jurusan"][7], tabel_hasil["Jurusan"][8], tabel_hasil["Jurusan"][9], tabel_hasil["Jurusan"][10], f'{nilai_test["Jurusan Diterima"][test_index]} - {nilai_test["PTN Diterima"][test_index]}'))
    aw.append(
        [tabel_hasil["Jurusan"][i] for i in range(1,10)]
    )
    aw[-1].insert(len(aw[-1]), f'{nilai_test["Jurusan Diterima"][test_index]} - {nilai_test["PTN Diterima"][test_index]}')

In [ ]:
dictHasil = dict()
for i in range(1,len(aw[0])+1):
    if i != len(aw[0]):
        dictHasil[f"tebakan {i}"] = [aw[idx][i-1] for idx in range(len(aw))]
    else:
         dictHasil[f"asli"] = [aw[idx][i-1] for idx in range(len(aw))]

dfHasil = pd.DataFrame(dictHasil)

dfHasil.to_csv(f"Hasil Prediksi {len(aw[0])} rank unk.csv")

In [ ]:
hasil = open_tabel(f"Hasil Prediksi {len(aw[0])} rank unk.csv")
hasilBenar = list()

for i in hasil.index:
    for col in hasil.columns:
        asl = hasil["asli"][i].split()
        s = " "
        aseli = s.join(asl)
        if col != "asli":
            hsl = hasil[col][i].split()
            try:
                hsl.remove("(IPA)")
            except ValueError:
                hsl.remove("(IPS)")
            finally:
                s = " "
                if s.join(hsl) == aseli:
                    hasilBenar.append([hasil[col][i], aseli])

In [ ]:
dictBenar = {
        "tebakan": [hasilBenar[i][0] for i in range(len(hasilBenar))],
        "asli": [hasilBenar[i][1] for i in range(len(hasilBenar))],
    }

dfBenar = pd.DataFrame(dictBenar)

dfBenar.to_csv("Hasil Prediksi Benar.csv")

In [ ]:
idiw = pd.read_csv("Hasil Prediksi 10 rank unk.csv")

idiw

In [ ]:
adaw = pd.read_csv("Hasil Prediksi Benar.csv")

adaw